In [1]:
from nmm import Qobj,spost,spre,cumulant,OverdampedBath
from numpy import random
import jax.numpy as jnp

In [2]:
A=random.rand(30,30)
B=random.rand(30,30)
C=random.rand(30,30)

In [3]:
A1=Qobj(jnp.array(A))
B1=Qobj(jnp.array(B))
C1=Qobj(jnp.array(C))

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [4]:
def test(A,B):
    m=spre(A)*spost(B)-0.5*spre(A*B) -0.5*spost(A*B)
    return m.expm()

In [5]:
#%timeit test(A1,B1)

In [6]:
import jax

In [7]:
f=jax.jit(test)

In [8]:
#%timeit f(A1,B1)

In [9]:
import qutip as qt

In [10]:
A2=qt.Qobj(jnp.array(A)).to("CSR")
B2=qt.Qobj(jnp.array(B)).to("CSR")
C2=qt.Qobj(jnp.array(C)).to("CSR")

In [11]:
def test2(A,B):
    m=qt.spre(A)*qt.spost(B)-0.5*qt.spre(A*B) -0.5*qt.spost(A*B)
    return m.expm()

In [12]:
#%timeit test2(A2,B2)

In [13]:
from nmm import jaxcsolve2
from nmm.cumulant.cumulant_jax2 import Qobj as Qobj2
from qutip import UnderDampedEnvironment

In [14]:
A3=Qobj2(jnp.array([[1,0],[0,-1]]))
B3=Qobj2(jnp.array([[0,1],[1,0]]))

In [15]:
env=UnderDampedEnvironment(lam=0.1,gamma=1,w0=1,T=1)
bath=env.approximate("matsubara",Nk=10)

In [16]:
cum=jaxcsolve2(Hsys=A3,Qs=[B3],t=jnp.linspace(0,10,100),baths=[bath])

In [17]:
def expm_superoperator_at_time(time_val, instance):
    # Call the generator method at the given time
    superoperator = instance.generator(time_val)
    # Compute and return the exponentiated superoperator (a spre object)
    return superoperator.expm()

In [18]:
cum.generator(3).data.shape

/home/mcditoos/cumulant.py/nmm/cumulant/cumulant_jax2.py:667: UserWarning: Explicitly requested dtype <class 'jax.numpy.complex128'> requested in array is not available, and will be truncated to dtype complex64. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  jnp.array(jnp.inf, dtype=jnp.complex128) * jnp.sign(omega_diff) # Use the original fill value


(4, 4)

In [19]:
from jax import jacfwd, jacrev
import jax.numpy as jnp

# Assume my_instance is your initialized jaxcsolve object
# Assume Qobj, spre, spost, jaxcsolve classes are defined as in the last corrected version

# Define the function that returns the non-scalar output (the spre object)
# This is the same function structure as before, but we won't pass it to grad
def expm_superoperator_at_time_object(time_val, instance):
    # Call the generator method to get the superoperator (returns a spre object)
    superoperator = instance.generator(time_val)
    # Compute the exponentiated superoperator (returns a spre object)
    return superoperator.expm() # Returns a spre object (non-scalar)

# Create the Jacobian function using jacfwd or jacrev.
# We want the Jacobian with respect to the first argument (time_val), so argnums=0.
# The output structure will match the function's output (a spre object).
jac_wrt_time_fn_object = jacfwd(expm_superoperator_at_time_object, argnums=0)
# Or you could use:
# jac_wrt_time_fn_object = jacrev(expm_superoperator_at_time_object, argnums=0)


# Compute the derivative (Jacobian) at a specific time point
specific_time_point = jnp.array(0.5, dtype=cum.t.dtype) # Use appropriate dtype
# Pass the specific time point (JAX array) and the jaxcsolve instance
derivative_spre_object = jac_wrt_time_fn_object(specific_time_point, cum)

# derivative_spre_object is a spre object
# its .data attribute contains the matrix derivative d(expm(L*t))/dt
print("Time derivative of expm(Generator(t)) (as a spre object) with respect to t at t=0.5:")
print(derivative_spre_object)
print("\nThe matrix data of the derivative is:")
print(derivative_spre_object.data) # This is the (N*N, N*N) matrix derivative

# If you only needed the derivative of the *data* matrix itself, you could differentiate
# a function that directly returns the data:
# def expm_superoperator_data_at_time(time_val, instance):
#      superoperator = instance.generator(time_val)
#      expm_superop = superoperator.expm()
#      return expm_superop.data # Returns a JAX array (non-scalar)
# jac_data_fn = jacfwd(expm_superoperator_data_at_time, argnums=0)
# derivative_data_matrix = jac_data_fn(specific_time_point, my_instance)
# print("\nMatrix derivative of expm(Generator(t)).data with respect to t at t=0.5:")
# print(derivative_data_matrix) # This is the same (N*N, N*N) matrix

/home/mcditoos/cumulant.py/nmm/cumulant/cumulant_jax2.py:667: UserWarning: Explicitly requested dtype <class 'jax.numpy.complex128'> requested in array is not available, and will be truncated to dtype complex64. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  jnp.array(jnp.inf, dtype=jnp.complex128) * jnp.sign(omega_diff) # Use the original fill value


DEBUG: spre _tree_flatten called on instance 140417783624528. Type of self.data is <class 'jax._src.interpreters.ad.JVPTracer'>. Shape: (4, 4)
DEBUG: spre _tree_flatten called on instance 140416643017632. Type of self.data is <class 'jaxlib.xla_extension.ArrayImpl'>. Shape: (4, 4)
DEBUG: spre _tree_flatten called on instance 140416643925680. Type of self.data is <class 'jax._src.interpreters.batching.BatchTracer'>. Shape: (4, 4)


TypeError: Value '<object object at 0x7fb592419290>' with dtype object is not a valid JAX array type. Only arrays of numeric types are supported by JAX.

In [ ]:
rho=jnp.array([[1,0],[0,0]])
rho0=Qobj(rho)

In [ ]:
states=cum.evolution(rho0)

TypeError: Unexpected input type for array: <class 'nmm.utils.utils.Qobj'>

In [ ]:
from nmm import csolve
from qutip import Qobj as qt_Qobj
from qutip import sigmaz,sigmax

In [ ]:
cum2=csolve(Hsys=sigmaz(),Qs=[sigmax()],t=jnp.linspace(0,10,100),baths=[bath],cython=False,matsubara=True)
states2=cum2.evolution(qt_Qobj(rho))

In [ ]:
uno=[i[0,0].data.item() for i in states]
dos=[i[0,0] for i in states2]

In [ ]:
import matplotlib.pyplot as plt
plt.plot(cum.t,uno)
plt.plot(cum.t,dos,"--")

In [ ]:
curr=jax.grad(cum.generator)

In [ ]:
curr(3)